# Baseline Model - TMDB Movie Rating Prediction

Amaç: Filmlerin `vote_average` (oy ortalaması) skorunu,
sadece temel sayısal özellikleri kullanarak tahmin eden
ilk (baseline) modeli kurmak.


In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

df = pd.read_csv("/content/data/processed/movies_clean.csv")
df.head()


,budget,genres,id,keywords,popularity,revenue,runtime,title,vote_average,vote_count,release_year,genre_count,keyword_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",150.437577,2787965087,162.0,Avatar,7.2,11800,2009.0,4,21
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",139.082615,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,2007.0,3,16
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",107.376788,880674609,148.0,Spectre,6.3,4466,2015.0,3,7
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",112.312950,1084939099,165.0,The Dark Knight Rises,7.6,9106,2012.0,4,21
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",43.926995,284139100,132.0,John Carter,6.1,2124,2012.0,3,16


# Feature ve Target Ayrımı

Modelin öğrenmesi için iki şey gerekir:

X (features) → modelin karar verirken kullandığı değişkenler

y (target) → modelin tahmin etmeye çalıştığı değer

In [16]:
import ast

json_cols = ["genres", "keywords"]

def parse_json_to_list(x):
    if pd.isna(x) or x == "" or x == "[]":
        return []
    try:
        return ast.literal_eval(x)
    except:
        return []

for col in json_cols:
    df[col] = df[col].apply(parse_json_to_list)

df["genre_count"] = df["genres"].apply(len)
df["keyword_count"] = df["keywords"].apply(len)


### Feature ve hedef seçimi
y = film puanı

X = puanı tahmin etmek için kullanacağımız sayısal kolonlar

Bu adımda `vote_average` sütununu hedef değişken (y) olarak seçtim.  
Aşağıdaki sayısal sütunları modelde feature olarak kullanıyorum:

- budget  
- revenue  
- runtime  
- popularity  
- vote_count  
- release_year  
- genre_count  
- keyword_count  

Hepsi sayısal ve doğrudan modellemeye uygun.


In [17]:
# Hedef değişken (tahmin etmeye çalıştığımız şey)
y = df["vote_average"]

# Kullanacağımız sayısal feature'lar
feature_cols = [
    "budget",
    "revenue",
    "runtime",
    "popularity",
    "vote_count",
    "release_year",
    "genre_count",
    "keyword_count",
]

# Her ihtimale karşı: dataframe'de gerçekten var olanları al
feature_cols = [c for c in feature_cols if c in df.columns]

X = df[feature_cols]

X.head(), y.head()


(      budget     revenue  runtime  popularity  vote_count  release_year  \
 0  237000000  2787965087    162.0  150.437577       11800        2009.0   
 1  300000000   961000000    169.0  139.082615        4500        2007.0   
 2  245000000   880674609    148.0  107.376788        4466        2015.0   
 3  250000000  1084939099    165.0  112.312950        9106        2012.0   
 4  260000000   284139100    132.0   43.926995        2124        2012.0   
 
    genre_count  keyword_count  
 0            4             21  
 1            3             16  
 2            3              7  
 3            4             21  
 4            3             16  ,
 0    7.2
 1    6.9
 2    6.3
 3    7.6
 4    6.1
 Name: vote_average, dtype: float64)

### Train–test ayrımı

Veriyi %80 eğitim, %20 test olacak şekilde ayırdım.  
Bu sayede modeli `X_train, y_train` üzerinde eğitip,  
hiç görmediği `X_test` üzerinde performansını ölçüyorum.

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape


((3842, 8), (961, 8))

### Baseline model: RandomForestRegressor

İlk baseline model olarak `RandomForestRegressor` kullandım.

- `n_estimators = 200`: Ormanda 200 ağaç var.
- `random_state = 42`: Sonuçların tekrar üretilebilir olması için sabit tohum.

Modeli `X_train` üzerinde eğittim ve `X_test` üzerinde:

- MAE (Mean Absolute Error): **…**  
- R² (R-kare): **…**

Bu skorlar proje için ilk referans (baseline) performansımız olacak.  
Sonraki feature engineering ve model optimizasyon adımlarında  
bu skorları iyileştirmeyi hedefleyeceğiz.


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Basit bir random forest modeli
model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

# Modeli eğit
model.fit(X_train, y_train)

# Test setinde tahmin yap
y_pred = model.predict(X_test)

# Metrikler
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, r2


(0.5390941727367325, 0.6266375272211631)

### Feature önemleri

Random forest modelinin feature önemlerine baktığımda,
tahmini en çok etkileyen değişkenlerin:

- vote_count
- runtime
- popularity

olduğunu görüyorum. Bu bilgiler, sonraki aşamalarda
feature engineering yaparken hangi değişkenlere odaklanmam gerektiği
konusunda yol gösteriyor.


In [20]:
import pandas as pd

importance = model.feature_importances_
feat_names = X_train.columns

fi_df = pd.DataFrame({
    "feature": feat_names,
    "importance": importance
}).sort_values("importance", ascending=False)

fi_df


,feature,importance
4,vote_count,0.487621
2,runtime,0.151548
3,popularity,0.089716
0,budget,0.079642
5,release_year,0.074481
7,keyword_count,0.046751
1,revenue,0.039456
6,genre_count,0.030785
